# Day 4: SAT RESULTS DATA CLEANING & DATABASE INTEGRATION

**Author:** Alexander Kuhn

**Date:** January 19, 2026

# This notebook shows:

1.  Import Required Libraries
2.  Load Dataset
3.  Cleaning Raw Dataset
4.  Validation Checks and Cleaning/Filtering
5.  Converting missing values: pandas format → PostgreSQL format
6.  Insert PostgreSQL Database and Verfication

# 1. Import Required Libraries

In [1]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_batch

# 2. Load Dataset

In [2]:
# Load raw SAT results dataset
df = pd.read_csv('/Users/alex/Documents/Webbeet_VS_Code/day_4_datasets/sat-results.csv')

# Preview the dataset
display(df.head())

# Inspect the dataset
print(df.info())
display(df.describe(include='all'))

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,SAT Critical Readng Avg. Score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363,355,218160,x345,78%,2.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366,383,268547,x234,NaN,3.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370,377,236446,x123,NaN,3.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359,414,427826,x123,92%,4.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384,390,672714,x123,92%,2.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   DBN                              493 non-null    object 
 1   SCHOOL NAME                      493 non-null    object 
 2   Num of SAT Test Takers           493 non-null    object 
 3   SAT Critical Reading Avg. Score  493 non-null    object 
 4   SAT Math Avg. Score              493 non-null    object 
 5   SAT Writing Avg. Score           493 non-null    object 
 6   SAT Critical Readng Avg. Score   493 non-null    object 
 7   internal_school_id               493 non-null    int64  
 8   contact_extension                388 non-null    object 
 9   pct_students_tested              376 non-null    object 
 10  academic_tier_rating             402 non-null    float64
dtypes: float64(1), int64(1), object(9)
memory usage: 42.5+ KB
None


,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,SAT Critical Readng Avg. Score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
count,493,493,493,493,493,493,493,493.000000,388,376,402.000000
unique,478,478,175,164,177,163,164,NaN,3,3,NaN
top,13K605,GEORGE WESTINGHOUSE CAREER AND TECHNICAL EDUCA...,s,s,s,s,s,NaN,x345,78%,NaN
freq,3,3,58,58,58,58,58,NaN,144,134,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,562172.943205,NaN,NaN,2.564677
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,262138.627055,NaN,NaN,1.126443
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101855.000000,NaN,NaN,1.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,332013.000000,NaN,NaN,2.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,587220.000000,NaN,NaN,3.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,782993.000000,NaN,NaN,4.000000


# 3. Cleaning Raw Dataset


## 3.1 Remove Redundant / Invalid Columns

In [ ]:
# Drop duplicated SAT column
df = df.drop(columns=['SAT Critical Readng Avg. Score'])

## 3.2 Normalize Column Names

In [ ]:
# Standardize column names
df.columns = (
    df.columns
    .str.strip()  # Remove whitespace
    .str.lower()  # Convert to lowercase
    .str.replace(' ', '_', regex=True)  # Replace spaces with underscores
    .str.replace(r'[^\w]', '', regex=True)  # Remove non-alphanumeric characters
)

print(df.columns)

Index(['dbn', 'school_name', 'num_of_sat_test_takers',
       'sat_critical_reading_avg_score', 'sat_math_avg_score',
       'sat_writing_avg_score', 'internal_school_id', 'contact_extension',
       'pct_students_tested', 'academic_tier_rating'],
      dtype='object')


## 3.3 Type Conversion and Parsing

In [ ]:
# Columns expected to be numeric
numeric_cols = [
    'num_of_sat_test_takers',
    'sat_critical_reading_avg_score',
    'sat_math_avg_score',
    'sat_writing_avg_score'
]

# Convert SAT-related columns to numeric
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Clean percentage column to numeric
df['pct_students_tested'] = (
    df['pct_students_tested']
    .str.rstrip('%')
    .pipe(pd.to_numeric, errors='coerce')
)

In [6]:
# Inspect distribution of academic tier rating
df['academic_tier_rating'].value_counts()

academic_tier_rating
4.0    112
2.0    101
3.0     96
1.0     93
Name: count, dtype: int64

In [7]:
# Convert academic tier rating to nullable integer
df['academic_tier_rating'] = (
    pd.to_numeric(df['academic_tier_rating'], errors='coerce')
    .astype('Int64')
)

# Inspect dataframe structure after conversion
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dbn                             493 non-null    object 
 1   school_name                     493 non-null    object 
 2   num_of_sat_test_takers          435 non-null    float64
 3   sat_critical_reading_avg_score  435 non-null    float64
 4   sat_math_avg_score              435 non-null    float64
 5   sat_writing_avg_score           435 non-null    float64
 6   internal_school_id              493 non-null    int64  
 7   contact_extension               388 non-null    object 
 8   pct_students_tested             376 non-null    float64
 9   academic_tier_rating            402 non-null    Int64  
dtypes: Int64(1), float64(5), int64(1), object(3)
memory usage: 39.1+ KB


# 4. Validation Checks

In [8]:
# Identify invalid SAT scores outside valid range (200-800)
print(
    f'=== Invalid Math Scores ===\n'
    f'{df[(df["sat_math_avg_score"] < 200) | (df["sat_math_avg_score"] > 800)]["sat_math_avg_score"].value_counts()}'
)

print(
    f'=== Invalid Critical Reading Scores ===\n'
    f'{df[(df["sat_critical_reading_avg_score"] < 200) | (df["sat_critical_reading_avg_score"] > 800)]["sat_critical_reading_avg_score"].value_counts()}'
)

print(
    f'=== Invalid Writing Scores ===\n'
    f'{df[(df["sat_writing_avg_score"] < 200) | (df["sat_writing_avg_score"] > 800)]["sat_writing_avg_score"].value_counts()}'
)

# Validate percentage values
print(
    f'=== Valid pct_students_tested Values ===\n'
    f'{df[(df["pct_students_tested"] >= 0) & (df["pct_students_tested"] <= 100)]["pct_students_tested"].value_counts().head()}'
)

=== Invalid Math Scores ===
sat_math_avg_score
 999.0     2
 850.0     1
-10.0      1
 1100.0    1
Name: count, dtype: int64
=== Invalid Critical Reading Scores ===
Series([], Name: count, dtype: int64)
=== Invalid Writing Scores ===
Series([], Name: count, dtype: int64)
=== Valid pct_students_tested Values ===
pct_students_tested
78.0    134
85.0    125
92.0    117
Name: count, dtype: int64


## 4.1 Filter Valid SAT Records

In [9]:
# Retain only rows with valid SAT scores (200-800 range)
df_filtered = df.copy()

for col in [
    'sat_critical_reading_avg_score',
    'sat_math_avg_score',
    'sat_writing_avg_score'
]:
    df_filtered = df_filtered[(df_filtered[col] >= 200) & (df_filtered[col] <= 800)]

# Reset index after filtering
df_filtered = df_filtered.reset_index(drop=True)
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dbn                             430 non-null    object 
 1   school_name                     430 non-null    object 
 2   num_of_sat_test_takers          430 non-null    float64
 3   sat_critical_reading_avg_score  430 non-null    float64
 4   sat_math_avg_score              430 non-null    float64
 5   sat_writing_avg_score           430 non-null    float64
 6   internal_school_id              430 non-null    int64  
 7   contact_extension               341 non-null    object 
 8   pct_students_tested             325 non-null    float64
 9   academic_tier_rating            358 non-null    Int64  
dtypes: Int64(1), float64(5), int64(1), object(3)
memory usage: 34.1+ KB


## 4.2 Duplicate Detection and Removal

In [10]:
# Identify fully duplicated rows
df_full_duplicates = df_filtered[df_filtered.duplicated(keep=False)]
df_full_duplicates = df_full_duplicates.sort_values(by='dbn')

# Display sample duplicates for manual inspection
display(df_full_duplicates.head().sort_values(by='dbn'))

# Remove exact duplicates
df_filtered_unique = df_filtered.drop_duplicates()
df_filtered_unique.info()

,dbn,school_name,num_of_sat_test_takers,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
32,02M419,LANDMARK HIGH SCHOOL,62.0,390.0,399.0,381.0,166135,x123,78.0,2
423,02M419,LANDMARK HIGH SCHOOL,62.0,390.0,399.0,381.0,166135,x123,78.0,2
45,02M520,MURRY BERGTRAUM HIGH SCHOOL FOR BUSINESS CAREERS,264.0,407.0,440.0,393.0,892839,NaN,92.0,2
421,02M520,MURRY BERGTRAUM HIGH SCHOOL FOR BUSINESS CAREERS,264.0,407.0,440.0,393.0,892839,NaN,92.0,2
428,02M520,MURRY BERGTRAUM HIGH SCHOOL FOR BUSINESS CAREERS,264.0,407.0,440.0,393.0,892839,NaN,92.0,2


<class 'pandas.core.frame.DataFrame'>
Index: 416 entries, 0 to 415
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dbn                             416 non-null    object 
 1   school_name                     416 non-null    object 
 2   num_of_sat_test_takers          416 non-null    float64
 3   sat_critical_reading_avg_score  416 non-null    float64
 4   sat_math_avg_score              416 non-null    float64
 5   sat_writing_avg_score           416 non-null    float64
 6   internal_school_id              416 non-null    int64  
 7   contact_extension               331 non-null    object 
 8   pct_students_tested             313 non-null    float64
 9   academic_tier_rating            349 non-null    Int64  
dtypes: Int64(1), float64(5), int64(1), object(3)
memory usage: 36.2+ KB


## 4.3 Remove Non-Analytical / Non-Relational Columns

In [11]:
# Drop columns not required for SAT performance analysis
df_filtered_unique.drop(
    columns=[
        'internal_school_id',
        'contact_extension'
    ],
    inplace=True
)

df_filtered_unique.info()

<class 'pandas.core.frame.DataFrame'>
Index: 416 entries, 0 to 415
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dbn                             416 non-null    object 
 1   school_name                     416 non-null    object 
 2   num_of_sat_test_takers          416 non-null    float64
 3   sat_critical_reading_avg_score  416 non-null    float64
 4   sat_math_avg_score              416 non-null    float64
 5   sat_writing_avg_score           416 non-null    float64
 6   pct_students_tested             313 non-null    float64
 7   academic_tier_rating            349 non-null    Int64  
dtypes: Int64(1), float64(5), object(2)
memory usage: 29.7+ KB


/var/folders/pt/nb3p6xq53hg0fxnb974jfsmc0000gn/T/ipykernel_29141/4137618431.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_unique.drop(


# 5. Converting missing values: pandas format → PostgreSQL format

In [12]:
# Step 1: Convert pd.NA to Python None
df_cleaned = df_filtered_unique.copy()
df_cleaned = df_cleaned.replace({pd.NA: None})

# Step 2: Ensure Python-native types for database compatibility
for col in df_cleaned.columns:
    if pd.api.types.is_integer_dtype(df_cleaned[col]):
        df_cleaned[col] = df_cleaned[col].astype(object)  # Int64 -> Python int/None
    elif pd.api.types.is_float_dtype(df_cleaned[col]):
        df_cleaned[col] = df_cleaned[col].astype(object)  # float64 -> Python float/None

# Convert to list of tuples for batch insertion
records = list(df_cleaned.itertuples(index=False, name=None))

# 6. Insert PostgreSQL Database

In [15]:
try:
    # 1. Establish a secure connection to PostgreSQL
   
    conn = psycopg2.connect(
        dbname="neondb",
        user="neondb_owner",
        password="a9Am7Yy5r9_T7h4OF2GN",
        host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
        port="5432",
        sslmode="require")
    cur = conn.cursor()  # Create a cursor object for executing SQL commands
    
    print("✓ Database connection established")
    
    # 2. Create the SAT scores table if it doesn't exist
    
    cur.execute("""
        DROP TABLE IF EXISTS nyc_schools.alex_sat_scores CASCADE;
        
        CREATE TABLE IF NOT EXISTS nyc_schools.alex_sat_scores (
            dbn TEXT PRIMARY KEY,
            school_name TEXT,
            num_of_sat_test_takers INTEGER,
            sat_critical_reading_avg_score FLOAT,
            sat_math_avg_score FLOAT,
            sat_writing_avg_score FLOAT,
            pct_students_tested FLOAT,
            academic_tier_rating INTEGER
        );
    """)
    
    print("✓ Table created successfully")
    
    # 3. Prepare parameterized INSERT statement
    
    insert_query = """
        INSERT INTO nyc_schools.alex_sat_scores (
            dbn,
            school_name,
            num_of_sat_test_takers,
            sat_critical_reading_avg_score,
            sat_math_avg_score,
            sat_writing_avg_score,
            pct_students_tested,
            academic_tier_rating
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (dbn) DO UPDATE SET
            school_name = EXCLUDED.school_name,
            num_of_sat_test_takers = EXCLUDED.num_of_sat_test_takers,
            sat_critical_reading_avg_score = EXCLUDED.sat_critical_reading_avg_score,
            sat_math_avg_score = EXCLUDED.sat_math_avg_score,
            sat_writing_avg_score = EXCLUDED.sat_writing_avg_score,
            pct_students_tested = EXCLUDED.pct_students_tested,
            academic_tier_rating = EXCLUDED.academic_tier_rating;
    """
    
    # 4. Execute batch insertion
    
    execute_batch(cur, insert_query, records, page_size=100)
    
    print(f"✓ {len(records)} records inserted/updated successfully")
    
    # 5. Commit changes and close connection
  
    conn.commit()
    cur.close()
    conn.close()
    
    print("✓ Database connection closed")
    print("\n----- Data integration completed successfully -----")
    
except Exception as e:
    print(f"\n✗ Error occurred: {e}")
    if conn:
        conn.rollback()
        conn.close()
    raise

✓ Database connection established
✓ Table created successfully
✓ 416 records inserted/updated successfully
✓ Database connection closed

----- Data integration completed successfully -----


## 6.1 Verification Query

In [ ]:
# Verify data was inserted correctly
try:
    conn = psycopg2.connect(
        dbname="neondb",
        user="neondb_owner",
        password="a9Am7Yy5r9_T7h4OF2GN",
        host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
        port="5432",
        sslmode="require"
    )
    
    verification_query = """
        SELECT COUNT(*) as total_records,
               AVG(sat_math_avg_score) as avg_math_score,
               AVG(sat_critical_reading_avg_score) as avg_reading_score,
               AVG(sat_writing_avg_score) as avg_writing_score
        FROM nyc_schools.alex_sat_scores;
    """
    
    df_verification = pd.read_sql_query(verification_query, conn)
    
    # Round numeric columns to 3 decimal positions
    
    df_verification = df_verification.round(3)
    
    display(df_verification)
    
    conn.close()
    
except Exception as e:
    print(f"Verification error: {e}")

/var/folders/pt/nb3p6xq53hg0fxnb974jfsmc0000gn/T/ipykernel_29141/956202472.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_verification = pd.read_sql_query(verification_query, conn)


,total_records,avg_math_score,avg_reading_score,avg_writing_score
0,416,413.733,401.067,394.175
